In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import Global_Functions as gf
import matplotlib.pyplot as plt

import Neuronal_Networks as nn

In [2]:
import time
timestr = time.strftime("%Y-%m-%d_%H-%M_")

In [3]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [4]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [5]:
train = "4"
val = "21"

ex_train = ex_4
ex_val = ex_21

In [6]:
NEUR = 128
EPOCH = 100
LAG = 60

In [7]:
image_path = "../Images/"
image_subfolder = image_path + "Intermediate Variable/"
specs = "training_{0} val_{1} {2}_neurons__{3}_epochs__lag_{4}" 
image_folder = image_subfolder + timestr + specs.format(train, val, NEUR, EPOCH, LAG) + "/"

gf.check_folder(image_path)
gf.check_folder(image_subfolder)
gf.check_folder(image_folder)

Folder already exists.
Folder already exists.
Creation of directory ../Images/Intermediate Variable/2021-03-19_09-32_training_4 val_21 128_neurons__100_epochs__lag_60/ successful.


In [8]:
np.random.seed(123)

In [9]:
def train_model(experiment_train, experiment_validation, lag_input=1, lag_output=1, batch_size = 1, nmb_epochs = 20, neurons = 64):
    scaler_train, X_train, y_train = nn.prepare_data(experiment_train, lag_input = lag_input, all_lags = False, differences = 'add', hybrid = False)
    scaler_val, X_val, y_val = nn.prepare_data(experiment_validation, lag_input = lag_input, all_lags = False,  differences = 'add', hybrid = False)
    
    model = nn.fit_lstm(X_train, y_train, X_val, y_val, 1, nb_epochs = nmb_epochs, neurons = neurons)

    return model, scaler_train, X_train, y_train, scaler_val, X_val, y_val    

In [10]:
model, scaler_train, X_train, y_train, _, X_val, y_val = train_model(ex_train, ex_val, lag_input = LAG, nmb_epochs = EPOCH, neurons=NEUR)

C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Neuronal_Networks.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['difference'] = data.diff(lag_input)['el_power']
C:\Users\FlorianLeiser\anaconda3\lib\site-packages\pandas\core\series.py:1042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-9-e65e3c5066ed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

9792/9795 [============================>.] - ETA: 0s - loss: 0.0051WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0010s). Check your callbacks.


9795/9795 [==============================] - 21s 2ms/step - loss: 0.0053 - val_loss: 0.3085


In [11]:
def predictions(experiment, model, batch_size = 1, specs = ""):    
    scaler, X, y = nn.prepare_data(experiment, lag_input = LAG, all_lags = False, differences = 'add', hybrid = False)
    
    preds_scaled = model.predict(X, batch_size = batch_size)
    preds = scaler[1].inverse_transform(preds_scaled)
    
    fig = plt.figure(figsize = (15,10))
    plt.plot(experiment['el_power'], color = gf.get_color("grey"), label = "True")
    plt.plot(preds, color = gf.get_color("green"), label = "Predictions")
    plt.ylabel('Electric power in [W]')
    plt.xlabel('Time')
    plt.legend()
    plt.title('Comparison predictions to true values' + specs, fontsize = 14)
    fig.tight_layout()
    plt.show()
    fig.savefig(image_folder + specs + "predictions.png")
    fig.savefig(image_folder + specs + "predictions.svg")
    
    return scaler, X, y, preds_scaled, preds

In [12]:
scaler_1, X_1, y_1, preds_scaled_1, preds_1 = predictions(ex_1, model, specs = " on ex_1 with model trained on " + train)

C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Neuronal_Networks.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['difference'] = data.diff(lag_input)['el_power']
C:\Users\FlorianLeiser\anaconda3\lib\site-packages\pandas\core\series.py:1042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-11-a833e0a1dde6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

AttributeError: 'tuple' object has no attribute 'predict'

In [ ]:
scaler_4, X_4, y_4, preds_scaled_4, preds_4 = predictions(ex_4, model, specs = " on ex_4 with model trained on " + train)

In [ ]:
scaler_9, X_9, y_9, preds_scaled_9, preds_9 = predictions(ex_9, model, specs = " on ex_9 with model trained on " + train)

In [ ]:
scaler_20, X_20, y_20, preds_scaled_20, preds_20 = predictions(ex_20, model, specs = " on ex_20 with model trained on " + train)

In [ ]:
scaler_21, X_21, y_21, preds_scaled_21, preds_21 = predictions(ex_21, model, specs = " on ex_21 with model trained on " + train)

In [ ]:
scaler_22, X_22, y_22, preds_scaled_22, preds_22 = predictions(ex_22, model, specs = " on ex_22 with model trained on " + train)

In [ ]:
scaler_23, X_23, y_23, preds_scaled_23, preds_23 = predictions(ex_23, model, specs = " on ex_23 with model trained on " + train)

In [ ]:
scaler_24, X_24, y_24, preds_scaled_24, preds_24 = predictions(ex_24, model, specs = " on ex_24 with model trained on " + train)